In [7]:
import pymongo
import os
import json
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
_URI = f"mongodb://{os.environ['MONGODB_USERNAME']}:{os.environ['MONGODB_PASSWORD']}@{os.environ['MONGODB_HOST']}:{os.environ['MONGODB_PORT']}/{os.environ['MONGODB_DATABASE']}"

In [9]:
client = pymongo.MongoClient(_URI)
client.list_database_names()

['admin', 'config', 'function_calling', 'local', 'mqtt_logs']

In [10]:
db = client["function_calling"]
collection = db["gorilla_openfunctions"]

In [33]:
import ast
def recursively_dump_to_json(input_json_str):
    # Parse the outer JSON string
    input_json_str = str(input_json_str)
    input_json_str = input_json_str.strip()
    
    try:
        outer_data = json.loads(input_json_str)
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}")
        print(f"Input JSON: {input_json_str}")
        return {
            "error": str(e),
            "data": input_json_str
        }

    # Function to recursively convert inner JSON strings
    def convert_inner_json(obj):
        if isinstance(obj, str):
            obj = obj.strip()
            if (obj.startswith("{") and obj.endswith("}")) or (obj.startswith("[") and obj.endswith("]")):
                try:
                    return ast.literal_eval(obj)
                except Exception as e:
                    print(f"Error parsing: {e}")
                    print(f"Input string: {obj}")
                    raise e
            
            return obj
            
        elif isinstance(obj, list):
            return [convert_inner_json(item) for item in obj]
        
        elif isinstance(obj, dict):
            return {key: convert_inner_json(value) for key, value in obj.items()}

    # Recursively convert inner JSON strings
    converted_data = convert_inner_json(outer_data)
    
    return dict(converted_data)

In [37]:
with open(os.environ["TEST_FILE_PATH"]) as f:
    for num, data in enumerate(f.readlines()):
        
        if data.strip() == "":
            continue

        data = recursively_dump_to_json(data)
        
        try:
            collection.insert_one(
                {
                    "split": "test",
                    "line": num+1,
                    "data": data
                }
            )
        except pymongo.errors.DuplicateKeyError:
            print(f"Duplicate key error at line {num+1}")
            # Get the duplcate element
            duplicate_element = collection.find_one({"split": "test", "line": num+1})
            if duplicate_element["data"] != data:
                print("Duplicate element is different")
                print(f"Old element: {duplicate_element}")
                print(f"New element: {data}")
                print("Deleting old element and inserting new element")
                collection.delete_one({"split": "test", "line": num+1})
                collection.insert_one(
                    {
                        "split": "test",
                        "line": num+1,
                        "data": data
                    }
                )
            else:
                print("Duplicate element is same, skipping")

In [38]:
with open(os.environ["TRAIN_FILE_PATH"]) as f:
    for num, data in enumerate(f.readlines()):
        
        if data.strip() == "":
            continue

        data = recursively_dump_to_json(data)
        
        try:
            collection.insert_one(
                {
                    "split": "train",
                    "line": num+1,
                    "data": data
                }
            )
        except pymongo.errors.DuplicateKeyError:
            print(f"Duplicate key error at line {num+1}")
            # Get the duplcate element
            duplicate_element = collection.find_one({"split": "test", "line": num+1})
            if duplicate_element["data"] != data:
                print("Duplicate element is different")
                print(f"Old element: {duplicate_element}")
                print(f"New element: {data}")
                print("Deleting old element and inserting new element")
                collection.delete_one({"split": "test", "line": num+1})
                collection.insert_one(
                    {
                        "split": "train",
                        "line": num+1,
                        "data": data
                    }
                )
            else:
                print("Duplicate element is same, skipping")

In [39]:
with open(os.environ["TRAIN_FILE_PATH"]) as f:
    data = f.readlines()

In [54]:
data[5467].strip()

'{"Instruction": "Can you provide me with information about a basketball tournament with id 200?\\n", "Functions": ["{\'name\': \'RapidAPI\', \'api_name\': \'requests.get\', \'description\': \'Broadage Basketball API can deliver any type of data for a basketball match, tournament or team; including livescores, play-by-play, boxscore, match statistics & many more. Our Basketball Coverage includes 100+ tournaments from all around the world with in-depth coverage, giving you the opportunity to present the best sports data to users located anywhere.  <br>This is a limited version in RapidApi. <a href=\\"https://www.broadage.com/signup/api/free?utm_source=rapidapi&utm_medium=click&utm_campaign=soccer_api\\" target=\\u201d_blank\\u201d>Please, click here to start your Free Trial and try the endpoints with live data now!</a>\', \'parameters\': [{\'name\': \'tournamentid\', \'description\': \'The id of the tournament.\', \'type\': \'NUMBER\'}]}\\n", "{\'name\': \'git\', \'api_name\': \'git.ver

In [55]:
json.dumps(recursively_dump_to_json(data[5467]))

'{"Instruction": "Can you provide me with information about a basketball tournament with id 200?", "Functions": [{"name": "RapidAPI", "api_name": "requests.get", "description": "Broadage Basketball API can deliver any type of data for a basketball match, tournament or team; including livescores, play-by-play, boxscore, match statistics & many more. Our Basketball Coverage includes 100+ tournaments from all around the world with in-depth coverage, giving you the opportunity to present the best sports data to users located anywhere.  <br>This is a limited version in RapidApi. <a href=\\"https://www.broadage.com/signup/api/free?utm_source=rapidapi&utm_medium=click&utm_campaign=soccer_api\\" target=\\u201d_blank\\u201d>Please, click here to start your Free Trial and try the endpoints with live data now!</a>", "parameters": [{"name": "tournamentid", "description": "The id of the tournament.", "type": "NUMBER"}]}, {"name": "git", "api_name": "git.version", "description": "Display version inf